In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('/Users/parthshukla/Documents/CollabProject/NBA-BoxScores-2024-2025.csv')
df

,Unnamed: 0,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0,22400061,1610612752,NYK,New York,1628384,OG Anunoby,OG,F,NaN,...,0.0,5.0,5.0,3.0,1.0,0.0,0.0,3.0,4.0,-21.0
1,1,22400061,1610612752,NYK,New York,1628404,Josh Hart,Josh,F,NaN,...,1.0,3.0,4.0,3.0,0.0,0.0,1.0,0.0,12.0,-23.0
2,2,22400061,1610612752,NYK,New York,1626157,Karl-Anthony Towns,Karl-Anthony,C,NaN,...,0.0,7.0,7.0,3.0,0.0,0.0,0.0,1.0,12.0,-18.0
3,3,22400061,1610612752,NYK,New York,1628969,Mikal Bridges,Mikal,G,NaN,...,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,16.0,-33.0
4,4,22400061,1610612752,NYK,New York,1628973,Jalen Brunson,Jalen,G,NaN,...,0.0,1.0,1.0,2.0,0.0,0.0,4.0,3.0,22.0,-23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,23,22400086,1610612757,POR,Portland,1641871,Duop Reath,Duop,NaN,DNP - Coach's Decision,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
732,24,22400086,1610612757,POR,Portland,1641712,Rayan Rupert,Rayan,NaN,DNP - Coach's Decision,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
733,25,22400086,1610612757,POR,Portland,1631101,Shaedon Sharpe,Shaedon,NaN,DND - Injury/Illness,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
734,26,22400086,1610612757,POR,Portland,1629680,Matisse Thybulle,Matisse,NaN,DND - Injury/Illness,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# dataset - https://www.kaggle.com/datasets/eoinamoore/historical-nba-data-and-player-box-scores/
teams_df = pd.read_csv('/Users/parthshukla/Documents/CollabProject/TeamStatistics.csv')

In [25]:
teams_df = teams_df[teams_df['gameDate'] > '2024-10-01']
teams_df

,gameId,gameDate,teamCity,teamName,teamId,opponentTeamCity,opponentTeamName,opponentTeamId,home,win,...,leadChanges,pointsFastBreak,pointsFromTurnovers,pointsInThePaint,pointsSecondChance,timesTied,timeoutsRemaining,seasonWins,seasonLosses,coachId
0,22400627,2025-03-19 21:00:00,Utah,Jazz,1610612762,Washington,Wizards,1610612764,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,32.0,NaN
1,22400627,2025-03-19 21:00:00,Washington,Wizards,1610612764,Utah,Jazz,1610612762,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,36.0,NaN
2,22400828,2025-02-24 22:00:00,Sacramento,Kings,1610612758,Charlotte,Hornets,1610612766,1,1,...,1.0,31.0,30.0,50.0,19.0,1.0,1.0,29.0,28.0,NaN
3,22400828,2025-02-24 22:00:00,Charlotte,Hornets,1610612766,Sacramento,Kings,1610612758,0,0,...,1.0,10.0,13.0,38.0,12.0,1.0,0.0,14.0,42.0,NaN
4,22400827,2025-02-24 21:00:00,Portland,Trail Blazers,1610612757,Utah,Jazz,1610612762,0,1,...,8.0,15.0,23.0,54.0,18.0,4.0,0.0,25.0,33.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1843,12400004,2024-10-05 19:00:00,Los Angeles,Clippers,1610612746,Golden State,Warriors,1610612744,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1844,12400003,2024-10-04 22:30:00,Los Angeles,Lakers,1610612747,Minnesota,Timberwolves,1610612750,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1845,12400003,2024-10-04 22:30:00,Minnesota,Timberwolves,1610612750,Los Angeles,Lakers,1610612747,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1846,12400001,2024-10-04 12:00:00,Boston,Celtics,1610612738,Denver,Nuggets,1610612743,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import warnings

from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score


In [27]:
team1 = input("Enter Team 1:")
team2 = input("Enter Team 2:")

team1_df = teams_df[(teams_df['teamName'] == team1) & (teams_df['opponentTeamName'] == team2)]
team1_df


,gameId,gameDate,teamCity,teamName,teamId,opponentTeamCity,opponentTeamName,opponentTeamId,home,win,...,leadChanges,pointsFastBreak,pointsFromTurnovers,pointsInThePaint,pointsSecondChance,timesTied,timeoutsRemaining,seasonWins,seasonLosses,coachId
1347,22400013,2024-11-15 19:30:00,Chicago,Bulls,1610612741,Cleveland,Cavaliers,1610612739,0,0,...,4.0,11.0,14.0,40.0,12.0,2.0,0.0,5.0,8.0,NaN
1399,22400212,2024-11-11 20:00:00,Chicago,Bulls,1610612741,Cleveland,Cavaliers,1610612739,1,0,...,17.0,18.0,11.0,48.0,15.0,15.0,0.0,4.0,7.0,NaN
1715,12400070,2024-10-18 20:00:00,Chicago,Bulls,1610612741,Cleveland,Cavaliers,1610612739,1,1,...,14.0,19.0,19.0,58.0,12.0,7.0,0.0,3.0,2.0,NaN
1821,12400016,2024-10-08 19:00:00,Chicago,Bulls,1610612741,Cleveland,Cavaliers,1610612739,0,1,...,5.0,31.0,31.0,68.0,19.0,3.0,0.0,1.0,0.0,NaN
